In [5]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations
from chains import BNBProcessor, BNBQuerier
import json
import web3



In [6]:

sql_db = SQLDatabase()
querier = BNBQuerier()
mongo_db = MongoDatabase()
processor = BNBProcessor(sql_db, mongo_db, querier)



In [7]:
query = "SELECT * FROM evm_contract_abis WHERE network = 'BNB'"
sql_db.cursor.execute(query)
contract_abis = sql_db.cursor.fetchall()


In [8]:
contract_abis[100]

RealDictRow([('network', 'BNB'),
             ('contract_address',
              '0x23c321EA0a151F1D4B670bf3B86B5D89e2b13da2'),
             ('abi',
              '[{"inputs": [], "stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "owner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "spender", "type": "address"}, {"indexed": false, "internalType": "uint256", "name": "value", "type": "uint256"}], "name": "Approval", "type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "previousOwner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "newOwner", "type": "address"}], "name": "OwnershipTransferred", "type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, "internalType": "address", "name": "from", "type": "address"}, {"indexed": true, "internalType": "address", "name": "to", "type": "addr

In [9]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from functools import partial

async def process_all_contracts():
    # Create a thread pool - adjust max_workers as needed
    thread_pool = ThreadPoolExecutor(max_workers=10)
    loop = asyncio.get_event_loop()
    
    # Create tasks that run in thread pool
    tasks = [
        loop.run_in_executor(
            thread_pool,
            partial(asyncio.run, processor._process_contract(contract['contract_address'], json.loads(contract['abi'])))
        )
        for contract in contract_abis
    ]
    
    # Gather results
    results = await asyncio.gather(*tasks)
    contract_info = [result for result in results if result]
    for result in contract_info:
        print(result)
    
    thread_pool.shutdown()
    return contract_info

# Run the async function

In [10]:
contract_info = await process_all_contracts()

Error processing contract <web3._utils.datatypes.Contract object at 0x000001BB802914D0>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x000001BB93842310>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x000001BB9C7C18D0>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x000001BBEA5B0550>: ('The abi for this contract contains no function definitions. ', 'Are you sure you provided the correct contract abi?')
Error processing contract <web3._utils.datatypes.Contract object at 0x000001BB9C810410>: cannot access local variable 'token1_info' where it is not associated with 

ContractInfo(address='0x73Eda54A43c6a3d08ee30B089Ef4C1bC3D0d4d9B', factory='0xcA143Ce32Fe78f1f7019d7d551a6402fC5350c73', fee=None, token0_name='Wrapped BNB', token1_name='Cornucopias [via ChainPort.io]', name=None)
ContractInfo(address='0xA8967Cd88AD3134474A8b1C485c1194C0e861DCd', factory='0xcA143Ce32Fe78f1f7019d7d551a6402fC5350c73', fee=None, token0_name='UltraSOL', token1_name='RocketFi', name=None)
ContractInfo(address='0x182135ab679292E90857D1D3a1Ad0A22D3dB317C', factory='0xcA143Ce32Fe78f1f7019d7d551a6402fC5350c73', fee=None, token0_name='Kekius Maximus', token1_name='Wrapped BNB', name=None)
ContractInfo(address='0x7f819e08a23D1d2739C71Eca25b77e2c71A20888', factory='0xcA143Ce32Fe78f1f7019d7d551a6402fC5350c73', fee=None, token0_name='MIN 128 Token', token1_name='Min Token', name=None)
ContractInfo(address='0xAe08C9D357731FD8d25681dE753551BE14C00405', factory='0x86407bEa2078ea5f5EB5A52B2caA963bC1F889Da', fee=None, token0_name='Tether USD', token1_name='SOLANA', name=None)
ContractIn